# 极简（大概）FCI

这一部分我们先以Hubbard模型为例子实现对Hubbard模型的简单FCI算法，因为Hubbard模型的FCI矩阵计算较为简单。

然后我们借用PySCF的部分代码，也实现一个较简单的对于分子体系二次量子化哈密顿量的FCI算法

## Hubbard模型简介
Hubbard模型是凝聚态物理中研究强关联电子系统的一个基础理论模型，由物理学家John Hubbard于1963年提出。它通过简化的数学框架描述了电子在晶格中的运动及其相互作用，是理解量子多体现象（如Mott绝缘体、高温超导等）的重要工具。

![2D Hubbard Model](./fig/2dhubbard.png "2D Hubbard Model")

Hubbard模型的哈密顿量如下所示：
$$
H = -t \sum_{\langle i, j\rangle, \sigma}\left(c_{i \sigma}^{\dagger} c_{j \sigma}+\text { h.c. }\right) + U \sum_i n_{i \uparrow} n_{i \downarrow}
$$

Hubbard 模型一般定义在一套格点上。其中第一项是动能项，只考虑近邻格点之间的相互作用。第二项是局域的电子-电子相互作用，Hubbard 模型只考虑了同一格点的自旋向上和自旋向下轨道都被占据时的电子-电子相互作用，这是一种对实际强关联系统的简化。

### 一维Hubbard模型的FCI算法
我们用最简单的一维Hubbard模型为例子介绍FCI方法。首先对一维Hubbard Model进行一下定义：

In [1]:
import numpy as np

# Hubbard模型定义：
norb = 6    # 格点数
nelec = 6   # 电子数
Sz = 0      # 自旋
U = 8       # Hubbard U 值
t = 1       # Hubbard t 值

nelec_a = (nelec+Sz) //2  # 自旋向上电子数
nelec_b = (nelec-Sz) //2  # 自旋向下电子数

h1 = np.zeros((norb,norb))  # 初始化单电子项
h2 = np.zeros((norb,norb,norb,norb))   # 初始化双电子项

# 一维 Hubbard Model 哈密顿量的定义： 
orb_list = np.arange(norb)
for orb_i in orb_list:
    # 对每个格点设置和相邻格点的相互作用
    orb_j = (orb_i + 1)%norb
    h1[orb_i,orb_j] = -1
    orb_j = (orb_i - 1)%norb
    h1[orb_i,orb_j] = -1

    # 每个格点还有Hubbard U项
    h2[orb_i,orb_i,orb_i,orb_i] = U

把h1 print出来看一下

In [2]:
h1

array([[ 0., -1.,  0.,  0.,  0., -1.],
       [-1.,  0., -1.,  0.,  0.,  0.],
       [ 0., -1.,  0., -1.,  0.,  0.],
       [ 0.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.,  0., -1.],
       [-1.,  0.,  0.,  0., -1.,  0.]])

然后就是写一个极简的FCI，计算一下这个哈密顿量的精确能量。

首先要弄清楚在这个希尔伯特空间里面到底有哪些组态。

根据我们前面的定义，有norb个格点，对应norb个轨道,2*norb个自旋轨道，其中norb个自旋向上轨道，norb个自旋向下轨道。有nelec个电子，其中nelec_a个自旋向上电子，nelec_b个自旋向下电子。可以通过排列组合得到所有可能的组态进行FCI计算。

In [3]:
from itertools import combinations

configurations = []
for spin_up_occ in combinations(range(norb), nelec_a):
    for spin_dn_occ in combinations(range(norb), nelec_b):
        configuration = np.zeros(2*norb) # 定义一个空组态，其长度为2*norb，等于自旋轨道数。自旋轨道的排列顺序为： 1↑ 1↓ 2↑ 2↓ 3↑ 3↓ ...
        spin_up_occ = np.array(spin_up_occ)
        spin_dn_occ = np.array(spin_dn_occ)
        configuration[2*spin_up_occ] = 1        # 根据spin_up_occ把占据的spin up obritals设置为1
        configuration[2*spin_dn_occ + 1] = 1    # 根据spin_dn_occ把占据的spin down obritals设置为1
        configurations.append(configuration)
configurations = np.array(configurations)

configurations里面就包括了所有组态了

我们可以将组态用一个二进制数来存储，比如可以把[1., 1., 1., 0., 0., 1., 0., 0.] 这个组态用 '0b100111' 这个二进制数进行存储，也可以进一步存储成十进制数(PySCF里面称之为ci_string或者string，我们沿用这个称呼)39，写一个把我们得到的configuration转换成十进制数(string)的函数：

In [4]:
def config_to_string(configurations:np.array):
    configurations = np.array(configurations,dtype = np.int64)
    n_configurations = configurations.shape[0] # 组态数
    n_spin_orbs = configurations.shape[1]   # 自旋轨道数
    strings = np.zeros(n_configurations, dtype = np.int64) # 先定义一个空的new_configurations的数组，里面用十进制数存组态
    for i in range(n_spin_orbs):
        strings += configurations[:,i]* 2**i # 每一位数值代表了2**i
    return strings

strings = config_to_string(configurations)
n_configurations = strings.shape[0]

获取了所有组态之后，就要计算组态之间的矩阵元了：
$$
H_{k,k^\prime} = \langle k | \hat{H} | k^\prime \rangle
$$

In [5]:
def cre_des_sign(p, q, string0):
    """
    这个函数的作用是，对于一个组态string0，它对于第p个轨道作用一个产生算符，第q个轨道作用一个湮灭算符，得到的新组态string1, 
    string0 和string1之间由交换反对称确定的符号是+1还是-1

    Args:
        p (int): 产生算符作用的位置
        q (int): 湮灭算符作用的位置
        string0 (int): 以十进制数保存的组套

    Returns:
        sign: +1 或者 -1
    """
    if p == q: 
        # 如果 p==q，那就是在同一个轨道上湮灭-产生，得到的组态是string0自己，这里不涉及费米子交换，符号为1
        return 1
    else:
        if (string0 & (1 << p)) or (not (string0 & (1 << q))):
            # 如果在p上本来有电子，或者q上没有电子，那么就无法产生/湮灭，这两个组态给出的矩阵元为0：
            return 0
        
        # 其他情况，则需要数一下产生算符p和湮灭算符q之间，交换了多少个电子，数一下p和q之间的1
        elif p > q:
            mask = (1 << p) - (1 << (q+1))
        else:
            mask = (1 << q) - (1 << (p+1))
        # 交换次数为偶数返回0， 为奇数返回1：
        return (-1) ** bin(string0 & mask).count('1')

def diagnol_matrix_term(ci_string: int, U: float) -> float:
    # Hubbard model 算双电子积分贡献的矩阵元比较方便，只需要数有多少个双占据就可以了，每有一个双占据就会贡献出一个Hubbard U的能量。
    sum_total = 0.0
    while ci_string > 0:
        if (ci_string & 3) == 3:  # 检查最后两位是否都是1
            sum_total += U
        ci_string >>= 2  # 右移两位，处理下一组
    return sum_total

def get_matrix_elements(ci_strings, annihilations, creations, U):
    """
    计算矩阵元的核心函数

    Args:
        ci_strings (np.array): 输入的组态数目
        annihilations (np.array): annihilations和creations记录了产生-湮灭对的信息。
        creations (np.array): 
        U (int): Hubbard U 的数值

    Returns:
        rows, cols, values: 非零矩阵元的行，列，值
    """
    n = len(ci_strings)
    str_dict = {s: idx for idx, s in enumerate(ci_strings)}  # 直接枚举索引
    
    # 预计算所有可能的（a, c）对
    operations = []
    for ann, cre in zip(annihilations, creations):
        for spin in (0, 1):
            a = ann * 2 + spin  # 湮灭位（spin-orbital）
            c = cre * 2 + spin  # 产生位（spin-orbital）
            operations.append((a, c))
    
    rows, cols, values = [], [], []
    
    for idx0, string_0 in enumerate(ci_strings):
        # 处理对角线元素
        rows.append(idx0)
        cols.append(idx0)
        values.append(diagnol_matrix_term(string_0, U))
        
        # 处理非对角线元素
        for a, c in operations:
            # 检查湮灭位是否为1且产生位是否为0
            if (string_0 & (1 << a)) and not (string_0 & (1 << c)):
                sign = cre_des_sign(c, a, string_0)
                if sign != 0:
                    string_1 = (string_0 & ~(1 << a)) | (1 << c)  # 安全位操作
                    if (idx1 := str_dict.get(string_1)) is not None:
                        rows.append(idx0)
                        cols.append(idx1)
                        values.append(sign * -1)
    rows = np.array(rows)
    cols = np.array(cols)
    values = np.array(values)
    return rows, cols, values

以上就是我们计算Hubbard model 矩阵元需要的函数。

接下来计算矩阵元，并构造哈密顿量矩阵。

In [6]:
annihilations, creations = h1.nonzero() # 首先把所有h1中记录的产生湮灭对记录一下
rows, cols, vals = get_matrix_elements(strings,annihilations,creations,U)

# 构造出矩阵
hamiltonian_matrix = np.zeros((n_configurations, n_configurations))
hamiltonian_matrix[rows,cols] = vals

In [7]:
hamiltonian_matrix

array([[24., -1.,  0., ...,  0.,  0.,  0.],
       [-1., 16., -1., ...,  0.,  0.,  0.],
       [ 0., -1., 16., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 16.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., 16.,  1.],
       [ 0.,  0.,  0., ...,  0.,  1., 24.]])

如果组态特别多的时候最好构建成稀疏矩阵：

In [8]:
import scipy.sparse
ham_coo_matrix = scipy.sparse.coo_matrix(\
            (vals, (rows, cols)), shape=(n_configurations,n_configurations)).astype(np.float64)

之后就可以对角化求特征值特征向量

In [9]:
import scipy.sparse.linalg
e,v = scipy.sparse.linalg.eigsh(ham_coo_matrix,which = "SA")
e[0]

-2.048130886091439

可以和PySCF的FCI对比一下

In [10]:
from pyscf.fci.direct_spin1 import FCI
myfci = FCI()
pyscf_fci_e , fcivec_pyscf = myfci.kernel(h1e = h1,eri = h2,norb = norb, nelec = nelec)
pyscf_fci_e - e[0]

2.842170943040401e-14